# Building a vacation planning assistance agent with smolagents

## Load a model from HuggingFace

In [ ]:
from smolagents import CodeAgent, HfApiModel
from tools.search_flight_tool import AmadeusFlightSearchTool
from tools.search_hotel_tool import AmadeusHotelFinderTool
from smolagents import FinalAnswerTool
import os
import dotenv
import yaml

# Should include your:
# - HF_API_KEY
# - AMADEUS_API_KEY
# - AMADEUS_API_SECRET

dotenv.load_dotenv()

HF_TOKEN = os.getenv("HF_API_KEY")

llm_model = HfApiModel(token=HF_TOKEN, model_id ="https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud")

## Build the travel agent with smolagents

In [ ]:
prompt_templates = yaml.safe_load(open("prompts.yaml", "r"))

agent = CodeAgent(
    model=llm_model,
    tools=[AmadeusFlightSearchTool(), AmadeusHotelFinderTool(), FinalAnswerTool()],
    planning_interval=3,
    max_steps = 5,
    additional_authorized_imports=["pandas", "requests", "dotenv", "re"],
    add_base_tools=False,
    prompt_templates=prompt_templates
)

## Adjust the system prompt

In [ ]:
system_instructions = """\n You're also an expert travel planning assistant! You can help users find flights between two destination cities, look for hotel offers in
the destination city and estimate costs to find the best possible option for the user. 

ALWAYS REMEMBER: 

- If there are no flights between the two cities, notify the user as final answer and ask for new instructions!
- If you do not have the necessary inputs based on user prompt to use a tool ALWAYS ask the user for the minimum input parameters that are needed and
  alert them for the existing options. Make sure you double check variables that have same names between subsequent tool calls to match what the user needs.
- ALWAYS LOOK FOR A RETURN FLIGHT UNLESS TOLD OTHERWISE BY THE USER!
- NEVER DO THINGS YOU ARE NOT ASKED TO DO! NEVER ADD unrealted costs derived BY ASSUMPTIONS unrelated to the user's request!
- If the user does not specify where they want to stay assume the city center(10 km radius) of the destination city.
- If a budget is specified it should be used for both the flight and the hotel.
- If some part of the trip is not covered, RETURN the part of the trip that is covered and alert the user in the final answer.

Don't hesitate to ask the user for confirmation of input information if something is not clear or missing.
Essential information the user SHOULD ALWAYS provided are, but not limited to:

- Departure city (City like "New York", never an airport name)
- Destination city (City like "London", never an airport name)
- Number of days to stay
- Travel date

If something of the above is not passed ask it as input.
If a suitable offer is found, provide it to the user DO NOT exhaustively look for better options as it will
overflow the API with requests.

Some examples on how to use the tools:

# flight_tool = AmadeusFlightSearchTool()
# flights = flight_tool.forward(
#     "London", "Great Britain", "New York", "United States of America", "2025-03-05"
# )
# print(flights)

# hotel_tool = AmadeusHotelFinderTool()
# hotels = hotel_tool.forward("London", 20, "Great Britain", 1, "2025-03-15", 1)
# print(hotels)

The final response tool should ALWAYS consist ONLY the following:

- The total cost of the trip (rounded to int)
- Flight details: All the best flight details for all outbound and optionally return flights (duration and departure time SHOUDL ALWAYS BE STATED for all flights!)
  Pick only one option as the best option for either flight or return flight!
- Accomodation details: The best hotel details (Pick only one option as the best option)

REMEMBER TO NEVER SIMULATE results or respond with fake data.You can ask the user to suggest alternative dates or respond with a message 
that the trip is not possible on those specific dates in case nothing fits the requested budget if one is provided or nothing was found in 
the search.
"""

# Update the system prompt
agent.system_prompt = agent.system_prompt + system_instructions

## Test the agent

In [ ]:
travel_planning_task = """I want to plan a trip to London from New York in the United States. I want to flight on the 23rd of March 2025. I want to stay for 3 days in London. Help me find the best option in terms of accomodation and flights."""

# Use the agent
response = agent.run(travel_planning_task)